In [1]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import AdamW
from torch.utils.data import DataLoader
import torch

In [2]:
# device = "cuda"
device = "cpu"

In [3]:
def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024 /1024
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("GPU 사용 불가")

In [4]:
def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id,cache_dir="./" )
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto",cache_dir="./", device_map=device)
    print_gpu_utilization()
    return model, tokenizer

In [5]:
model_id = "Qwen/Qwen2-0.5B-Instruct" #GPU-1G
#model_id = "Qwen/Qwen2-1.5B-Instruct" #GPU - 4G 
model, tokenizer = load_model_and_tokenizer(model_id)

GPU 메모리 사용량: 0.000 GB


In [7]:
prompt = """다음 문장에서 잘못된 문장을 수정해 줘"
'집행관리조직은 대체로 작은 규모이면서 최고정책결정자(총리, 대 통령 등)의 직접적 업무범위 내에 위치하거나 중앙정부의 중심(center of government)에 위치하고 있다. 
작은 규모의 조직은 상대적으로 신축적이 며 응집성이 강하지만 이와 함께 우수한 역량의 직원을 선발해야 하고, 관 주 등에서도 행정개혁의 일환이거나 이를 위한 pilot 프로그램으로 시도되었다.
32 Beattie가 이끄는 노동당의 2004년 재선 이후 초선 정부 시 나타난 주요 부처의 실망스 러운 성과를 반성하고 보다 견고한 정책제안을 내각에 제시하기 위해 설치되었다 (Tiernan[2006]).
33 2003년 Howard 수상에 의해 설치되었으며, 정부의 주요 정책들에 대한 집행 및 서비 스 전달을 촉진하는 역할과 범부처의 정책집행 진척과정을 보고하는 역할을 담당하였다 (Australian Department of Prime Minister and Cabinet[2013]).
34 미국의 경우에도 [그림 3]가 같이 Obama 2기 정부에서 High Priority Performance Goals라는 집행관 리를 시도하였는데, 여기서는 개별적인 집행관리부서를 두지 않고, 기존의 OMB가 그 역할을 대신하고 있다.
106 정책효과성 증대를 위한 집행과학에 관한 연구 리대상인 업무가 제한적⋅선택적이어야 한다는 제약이 있다.'"""
messages = [
    {"role": "system", "content": "You are a korean editor"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
response